In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scripts.read_land_use_no_gdal as rlu
import scripts.read_image_data_scaleable as rid

In [2]:
from importlib import reload
reload(rid)

<module 'scripts.read_image_data_scaleable' from '../scripts/read_image_data_scaleable.py'>

In [3]:
image_dir='../area3/2014/images/'
mask_dir='../area3/2014/masks/'
table_dir='../area3/2014/tables/LC8_SR.csv'
new_table_dir='../area3/2015/tables/LC8_SR.csv'
shelve_root_dir='../models/area3/shelves/'
processes=8
max_days_apart=30
step=250000

In [4]:
ds = rid.read_image_data(image_dir, mask_dir, table_dir, shelve_root_dir + 'old/', processes)

In [ ]:
res = ds[list(ds.keys())[0]].shape[1:]
new_table = pd.read_csv(new_table_dir)
new_times = list(new_table['system:time_start'])
if timestamps is None:
    times_to_fit = []
    for t in new_times:
        dt = datetime.datetime.fromtimestamp(t / 1000)
        dt = dt.replace(year=dt.year - 1)
        times_to_fit += [int(dt.timestamp() * 1000)]
else:
    times_to_fit = timestamps
times_to_fit.sort()

In [ ]:
imgs = rid.interpolate_images(times_to_fit, ds, max_days_apart, processes, shelve_root_dir)

In [ ]:
pix = rid.store_set(imgs, processes, shelve_root_dir + 'old/')
imgs.close()

In [3]:
# area3
polygon = "POLYGON ((144.0229424345357 -37.55299953752239, 144.02019905996406 -37.97282887730929, 144.6632499098556 -37.972670022747636, 144.66240633494579 -37.552454710108975, 144.0229424345357 -37.55299953752239))"
array, classes = rlu.read_land_use(da_shapefile="../SDM324649_full/ll_gda94/sde_shape/whole/VIC/CATCHMENTS/layer/landuse_2014.shp",
                  resolution=(2387, 1976),
                  area_filter=polygon,
                  buffer=-0.0001,
                  processes=8)

In [ ]:
train = rid.combine_set(pix, shelve_root_dir + 'old/', res, step, processes, array)